<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/Sklearn_Designing_New_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Classifer

In [0]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KernelDensity
import numpy as np

In [0]:
class KDEClassifier(BaseEstimator, ClassifierMixin):
  
  def __init__(self, bandwidth=1.0, kernel='gaussian'):
    self.bandwidth = bandwidth
    self.kernel = kernel
    
  def fit(self, X, y):
    self.classes_ = np.sort(np.unique(y))
    training_sets = [X[y == c] for c in self.classes_]
    
    self.models_    = [ KernelDensity(bandwidth=self.bandwidth, kernel=self.kernel).fit(X_i) for X_i in training_sets]
    self.logpriors_ = [ np.log(X_i.shape[0]/X.shape[0]) for X_i in training_sets]
    
  def predict_proba(self, X):
    logprobs = np.array([ model.score_samples(X) for model in self.models_]).T
    result = np.exp(logprobs + self.logpriors_)
    return result / result.sum(1, keepdims=True)
  
  def predict(self, X):
    return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [11]:
from sklearn.datasets import load_digits
from sklearn.model_selection import GridSearchCV

digits = load_digits()

bandwidths = 10 ** np.linspace(0, 2, 100)
grid = GridSearchCV(KDEClassifier(), {'bandwidth': bandwidths})
grid.fit(digits.data, digits.target)

print(grid.best_estimator_)
print(grid.best_params_)
print('accuracy =', grid.best_score_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KDEClassifier(bandwidth=7.054802310718643, kernel='gaussian')
{'bandwidth': 7.054802310718643}
accuracy = 0.9666110183639399
